<a href="https://colab.research.google.com/github/DLPY/Regression-Session-2/blob/master/Ridge_Lasso_Rgression_Lab2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Open In Colab

# House Price Prediction based on Postal Code, Number of Bathrooms, Car Parking and Property Type

Detail on Data: https://www.kaggle.com/mihirhalai/sydney-house-prices

# Download source data from Github
!wget https://raw.githubusercontent.com/DLPY/Regression-Session-2/master/Data/SydneyHousePrices.csv

# Import necessary packages for performing EDA and Multiple Regression

In [ ]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np 
import seaborn as sns
from sklearn.preprocessing import (LabelEncoder, OneHotEncoder, StandardScaler)
from sklearn.model_selection import train_test_split
from sklearn.linear_model import (LinearRegression, Ridge, Lasso)
from sklearn.metrics import (r2_score, mean_squared_error)
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

%matplotlib inline

pd.set_option('display.max_colwidth', None)

!wget https://raw.githubusercontent.com/DLPY/Regression-Session-2/master/Data/SydneyHousePrices.csv

# Read data from csv file into Pandas dataframe

In [ ]:
df = pd.read_csv('SydneyHousePrices.csv')
# df = pd.read_csv('SydneyHousePrices.csv')

# Exploratory Data Analysis (EDA)
Perform data analysis, cleaning and transformation.

## Data Analysis

In [ ]:
# Display the count of rows and columns.
df.shape

In [ ]:
# Review a small sample of the data.
df.head(5)

In [ ]:
# Review the data types.
df.dtypes

##### From the above, notice the different types of data: integer (int64), float (float64), and text (object).
---

### Review the date range.

Additional date-aggregation functions become available when the data type is converted to date format.  

The data type for date range is currently text, which is okay for now.

In [ ]:
print('Date ranges from {} to {}'.format(df.Date.min(), df.Date.max()))

### Review descriptive statistics of the numerical data.

In [ ]:
round(df.describe(), 2)

##### From the above max (bed & bath) and min (sellPrice) - It shows the dataset has outliers that need to be removed.
---

In [ ]:
# Detailed overview of the dataframe itself.
df.info()

##### From the above, notice that the Non-Null Count values are different for bed and car.
---

## Data Cleaning

### Explore the missing values

Review the count of non-null values for each column.  These need to be filled in prior to modelling.

In [ ]:
df.isnull().sum()

### Aggregate the missing values by date
Combining the method from above with the dataframe's groupby method to group missing car/bed values by date.

Plotting the output with the dataframe's plot method to see if there are any pockets of missing data.

In [ ]:
df.groupby(['Date'])['car'].apply(lambda x: x.isnull().sum()).plot()

In [ ]:
df.groupby(['Date'])['bed'].apply(lambda x: x.isnull().sum()).plot()

### Fill in the missing values
Assuming that houses in a particular area have a similar bath count, missing values can be filled in by:
 1. Grouping postal code and bath.
 2. Calculating the median value of bed and car for each of these groups.
 3. Filling in the missing values with the median count of each group.

In [ ]:
df['bed'].fillna(df.groupby(['postalCode', 'bath'])['bed'].transform('median'), inplace=True)

In [ ]:
df['car'].fillna(df.groupby(['postalCode', 'bath'])['car'].transform('median'), inplace=True)

### Remove outliers in the data
Using a function that removes the outliers from each column on the list, loop through a list of specific columns that were identified earlier in the EDA as having outliers.

The resulting dataframe does not contain any outliers.

In [ ]:
def remove_outlier(df_in, col_name):
    '''Removes outliers from a specified column of a dataframe using IQR and returns an updated dataframe.'''
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3 - q1 #Interquartile range
    fence_low  = q1 - 1.5 * iqr
    fence_high = q3 + 1.5 * iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [ ]:
outliers = ['bath', 'sellPrice', 'car', 'bed']

for outlier in outliers:
    df = remove_outlier(df, outlier).reset_index(drop=True)

## Transformation

The model must include some date related features in order to make better predictions.

New features can be created using the purchase dates.

 * Recent house prices are typically different from historic prices.
   - The date range of the model should be fairly recent, so date can be filtered to only include three years of the most recent data.
 * House prices tend to move slowly, on a monthly basis. 
   - A 'diffDate' feature can be created by calculating the difference of sale date and most recent date within the data.
 * There may be annual seasonality associated with house purchases.
   - A 'Quarter' feature can be created that bins the dates by annual quarter.

### Transform dates for analysis of sales type prices of property types by year, month.

In [ ]:
# Convert the 'Date' datatype to datetime so that Pandas date functions become available.
df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# Get the maximum purchase date.
max_date = df['Date'].max()
max_date

In [ ]:
# Filter the dataframe so that the most recent data is available, three years in this case.
df = df[df['Date'] >= (max_date - np.timedelta64(3, 'Y'))]

In [ ]:
df.shape

In [ ]:
# Create the 'diffDate' feature.
df['diffDate'] = df['Date'].apply(lambda x: max_date - x)

In [ ]:
# Alter 'diffDate' so that it captures the timeframe values as months.
df['diffDate'] = df['diffDate'] / np.timedelta64(1, 'M')

In [ ]:
# Alter 'diffDate' so that monthly values are ints, not floats. Floats would be equivalent to weekly values, too granular.
df['diffDate'] = df['diffDate'].astype(int)

In [ ]:
# The Pandas date type allows the date to be split in various ways: year, month, day, quarter.
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Quarter'] = df['Date'].dt.quarter

In [ ]:
# Plotting sellPrice by year and month.
fig, ax = plt.subplots(nrows=4, ncols=1, figsize=(15, 12))
sns.pointplot(data=df, x='Year', y='sellPrice', hue='propType', ax=ax[0], ci=None)
sns.pointplot(data=df, x='Month', y='sellPrice', hue='propType', ax=ax[1], ci=None)
sns.pointplot(data=df, x='Day', y='sellPrice', hue='propType', ax=ax[2], ci=None)
sns.pointplot(data=df, x='Quarter', y='sellPrice', hue='propType', ax=ax[3], ci=None)

##### From the above, notice the overall seasonal and annual trends of each property type.
---

Create a new dataframe that captures the median sale price value of similar sized property types within each postal code and suburb.

In [ ]:
medSellPrice = df.groupby(['postalCode', 'suburb', 'bath', 'car', 'bed'])['sellPrice'].apply(lambda x: x.median()).reset_index()

In [ ]:
# Rename the column as 'medSellPrice'
medSellPrice = medSellPrice.rename(columns={'sellPrice': 'medSellPrice'})
medSellPrice[:5]

##### From the above, notice the median sale price for each property size are different even when the postal code and suburb are the same.
---

In [ ]:
# Merge the dataframes based on property similarity, postalCode and sellPrice
df = pd.merge(df, medSellPrice, how='outer', on=['postalCode', 'suburb', 'bath', 'car', 'bed'])

In [ ]:
df.head()

### Encoding the categorical variables - Change the text into numbers

Review the unique values within property type.

In [ ]:
df.propType.unique()

In [ ]:
# Drop warehouse, acreage, and other property types in order to focus specifically on housing data.
df = df.drop(df[(df.propType == 'warehouse') | (df.propType == 'acreage') | (df.propType == 'other')].index)

Convert the property type values into numeric categorical labels so that this data can be used in the model.

In [ ]:
df['propType'] = df['propType'].astype('category').cat.codes

In [ ]:
df.head(5)

##### From the above, notice that:
 * diffDate is a numeric value representing the approximate count of months from sale date to July 6, 2019, (the max date in the data).
 * The original date column has been split into new columns: Year, Month, Day, Quarter, diffDate, and medSellPrice.
 * The propType categories have been converted to a numeric value. Warehouse, acreage, and other property types have been removed to specifically focus on housing.
 ---

### Quick review - columns that are not useful and need to be dropped:
* **Date** - diffDate will be used instead.
* **Id** - This is simply a row number of the data.
* **suburb** - postalCode will be used instead because it is a more generalised representation of locality.
* **Year, Month, Day** - Quarter will be used because it is a more generalised representations of Date. 

# Choosing predictors and target variables for performing Multiple Regression
**Target and Source variables**

* **Target Variable:** sellingPrice
* **Predictor Variables:** ordDate, postalCode, bed, bath, car, propType

### Create a new dataframe that includes only the selected columns

In [ ]:
df_new = df[['Quarter', 'diffDate', 'medSellPrice', 'postalCode', 'bed', 'bath', 'car', 'propType', 'sellPrice']]

In [ ]:
df_new.head(5)

# Investigate correlation in the new dataframe.

Pandas has a built-in correlation function.

In [ ]:
corr = df_new.corr()

In [ ]:
# View the correlations within the dataframe.
corr

Create a heatmap chart of the correlations, using a mask to hide redundant information and correct aspect ratio to ensure proper spacing of the chart.

In [ ]:
mask = np.zeros_like(corr, dtype=bool)
mask[np.triu_indices_from(mask)] = True
cmap = sns.diverging_palette(10, 250, as_cmap=True)
f, ax = plt.subplots(figsize=(11, 9))
sns.heatmap(corr, mask=mask, cmap=cmap, square=True,
            linewidths=0.2, cbar_kws={'shrink': 0.5}, ax=ax, annot=True)

##### From the above; notice the medium correlation of bed and bath, the negative medium correlation of postCode and sellPrice.  

Multicollinearity has a negative impact on multiple regression models.  The steps for overcoming multicollinearity are different for the chosen model.

* **Training a model and testing on unseen data:** standardize the features.
* **Multiple Regression on entire data set:** a variance inflation factor (VIF) analysis will be used for feature selection.

Either way, the target and predictor variables must be split into different dataframes prior to modelling.

# Isolate Target and Predictor Variables to Different Dataframes

In [ ]:
X = df_new[['Quarter', 'diffDate', 'medSellPrice', 'postalCode', 'bed', 'bath', 'car', 'propType']]
y = df_new[['sellPrice']]

# Save this list of column values for later
columns_list = list(X.columns.values)

In [ ]:
X.head(5)

In [ ]:
y.head(5)

# Standardise Features

Because in linear regression the value of the coefficients is partially determined by the scale of the feature, and in regularized models all coefficients are summed together, the features must be standardised prior to training.

The approach to standardising features is removing the mean and scale to unit variance.

The standard score of a sample x is calculated as:

    z = (x - u) / s

where _u_ is the mean of the training samples and _s_ is the standard deviation of the training samples.

In [ ]:
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

In [ ]:
# The scaled values are now stored as an array.
X_std[: 5]

In [ ]:
# X is already an array data type, so y also needs converting (the model expects these as inputs).
y = y.values

# Split dataset into the training and test using train_set_split: 

90% - train

10% - test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_std, y, test_size=0.1, random_state=23)

In [ ]:
print('Training Data:', X_train.shape, y_train.shape)
print('Testing Data:', X_test.shape, y_test.shape)

# Train, Test and Predict using ridge and lasso regression models

## Multiple Regression

In [ ]:
multipleregressor = LinearRegression()

Fit the linear regression model to the training set. We use the fit method the arguments of the fit method will be training sets

In [ ]:
multipleregressor.fit(X_train, y_train)

Regression Coefficients

In [ ]:
regression_coefficients = pd.DataFrame(multipleregressor.coef_)
regression_coefficients.columns = columns_list
regression_coefficients

### Prediction and Evaluation metrics - How to Calculate R-Square and RMSE

In [ ]:
y_pred = multipleregressor.predict(X_test)

print('Multiple Regression results:')
coefficient_of_dermination_reg = r2_score(y_test, y_pred)
print('R-squared: {}'.format(coefficient_of_dermination_reg))

rmse_reg = np.sqrt(mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error: {}'.format(rmse_reg))

In [ ]:
# Displaying Results and Difference in Table 
res = pd.DataFrame(y_pred, y_test.ravel())
res = res.reset_index()
res.columns = ['Price', 'Prediction']
res['Prediction'] = round(res['Prediction'], 0)
res['Difference'] = res['Prediction'] - res['Price']
res.head(5)

## Ridge Regression
Create an object called RidgeRegression in the regression class with alpha 0.01

In [ ]:
ridgeregressor = Ridge(alpha=0.01)

Fit the ridge regression model to the training set. We use the fit method the arguments of the fit method will be training sets

In [ ]:
ridgeregressor.fit(X_train, y_train)

Regression Coefficients

In [ ]:
ridge_coefficients = pd.DataFrame(ridgeregressor.coef_)
ridge_coefficients.columns = columns_list
ridge_coefficients

Training set prediction score

In [ ]:
y_pred = ridgeregressor.predict(X_train)

In [ ]:
ridge_train_score = ridgeregressor.score(X_train, y_train)
ridge_train_score

Training set prediction score

In [ ]:
y_pred = ridgeregressor.predict(X_test)

In [ ]:
ridge_test_score = ridgeregressor.score(X_test, y_test)
ridge_test_score

##### From the above, notice that the results of the test data are slightly better than the results of the training data (higher score is better).
This suggests that the model is generalised enough to work well with previously unseen data.

### Evaluation metrics - How to Calculate R-Square and RMSE

In [ ]:
print('Ridge results:')
coefficient_of_dermination_ridge = r2_score(y_test, y_pred)
print('R-squared: {}'.format(coefficient_of_dermination_ridge))

rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error: {}'.format(rmse_ridge))

##### From the above, notice that the results of the R-Square score are the same as the prediction score on the test set.

In [ ]:
# Displaying Results and Difference in Table 
res = pd.DataFrame(y_pred, y_test.ravel())
res = res.reset_index()
res.columns = ['Price', 'Prediction']
res['Prediction'] = round(res['Prediction'], 0)
res['Difference'] = res['Prediction'] - res['Price']
res.head(5)

In [ ]:
res['Difference'].median()

## Create an object called RidgeRegression in the regression class with alpha 100

In [ ]:
ridgeregressor100 = Ridge(alpha=100)
ridgeregressor100.fit(X_train, y_train)

In [ ]:
ridge100_coefficients = pd.DataFrame(ridgeregressor100.coef_)
ridge100_coefficients.columns = columns_list
ridge100_coefficients

In [ ]:
print('Ridge100 results:')
y_pred = ridgeregressor100.predict(X_test)

coefficient_of_dermination_ridge100 = r2_score(y_test, y_pred)
print('R-squared:', coefficient_of_dermination_ridge100)

rmse_ridge100 = np.sqrt(mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error: {}'.format(rmse_ridge100))

In [ ]:
Ridge_train_score_100 = ridgeregressor100.score(X_train, y_train)
Ridge_train_score_100

In [ ]:
Ridge_test_score_100 = ridgeregressor100.score(X_test, y_test)
Ridge_test_score_100

In [ ]:
# Displaying Results and Difference in Table 
res = pd.DataFrame(y_pred, y_test.ravel())
res = res.reset_index()
res.columns = ['Price', 'Prediction']
res['Prediction'] = round(res['Prediction'], 0)
res['Difference'] = res['Prediction'] - res['Price']
res.head(5)

In [ ]:
res['Difference'].median()

## Create an object called Lasso in the regression class with alpha 1

In [ ]:
# TODO: Should explain here that Python uses the term Alpha...

In [ ]:
lassoregressor = Lasso(alpha=1, max_iter=10e6)
lassoregressor.fit(X_train, y_train)
print('Coefficients: ', lassoregressor.coef_)

In [ ]:
lasso_coefficients = pd.DataFrame(lassoregressor.coef_).T
lasso_coefficients.columns = columns_list
lasso_coefficients

In [ ]:
print('Lasso results:')
y_pred= lassoregressor.predict(X_test)

coefficient_of_dermination_lasso = r2_score(y_test, y_pred)
print('R-squared:', coefficient_of_dermination_lasso)

rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error: {}'.format(rmse_lasso))

# Comparison of Model Outputs

In [ ]:
# Append the model coefficient outputs into a single dataframe.
df_coeff = ridge_coefficients.append([regression_coefficients,ridge100_coefficients, lasso_coefficients]).reset_index(drop=True)

In [ ]:
df_coeff

In [ ]:
# Create a list of models, their r-squared and rmse values.
model_results = [('multiple regression', coefficient_of_dermination_reg, rmse_reg),
                 ('ridge', coefficient_of_dermination_ridge, rmse_ridge),
                 ('ridge100', coefficient_of_dermination_ridge100, rmse_ridge100),
                 ('lasso', coefficient_of_dermination_lasso, rmse_lasso)]

In [ ]:
# Create a dataframe to review the model results.
overall_results = pd.DataFrame(model_results, columns=['Model', 'R-Squared', 'RMSE'])

In [ ]:
overall_results

In [ ]:
overall_results.merge(df_coeff, how='outer', left_index=True, right_index=True)

##### From the above, notice that the overall results are similar. Ridge is the champion model due to the lowest RMSE and highest R-Squared values.
---

Plotting the coefficients of each model displays that the values of each model's coefficients are also quite similar.

In [ ]:
# Note: is this too much information?
plt.plot(ridgeregressor.coef_.ravel(), alpha=0.7, linestyle='none', marker='*', markersize=5,
         color='red', label=r'Ridge; $\alpha = 0.01$', zorder=7) 
plt.plot(ridgeregressor100.coef_.ravel(), alpha=0.5, linestyle='none', marker='d', markersize=6,
         color='blue', label=r'Ridge; $\alpha = 100$') 
plt.plot(lassoregressor.coef_.ravel(), alpha=0.4, linestyle='none', marker='o', markersize=7,
         color='green', label='Lasso Regression')
plt.xlabel('Coefficient Index', fontsize=14)
plt.ylabel('Coefficient Magnitude', fontsize=14)
plt.legend(fontsize=13, loc=1)
plt.xticks(range(0, len(columns_list)), columns_list)
plt.show()

# Recap of outputs using these methods

TODO: Recap the model approaches, similarities/differences, and the intent.

---

# VIF

A simple method to detect multicollinearity in a model is by using something called the variance inflation factor or the VIF for each predicting variable.

VIF measures the ratio between the variance for a given regression coefficient with only that variable in the model versus the variance for a given regression coefficient with all variables in the model.

VIF starts at 1 and has no limits. A VIF of 1 (the minimum possible VIF) means the tested predictor is not correlated with the other predictors. A VIF of 1 (the minimum possible VIF) means the tested predictor is not correlated with the other predictors.

The higher the VIF:
* The more correlated a predictor is with the other predictors
* The more the standard error is inflated
* The larger the confidence interval
* The less likely it is that a coefficient will be evaluated as statistically significant

VIF = 1, _no correlation_ beetween independent variables. 

VIF > 10, _high multicollinearity_ between independent variables.

In [ ]:
# Create VIF dataframe and calculate VIF for each feature.
vif_data = pd.DataFrame()
vif_data['feature'] = X.columns.values
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(len(X.columns))]
vif_data

##### From the above, there are several multicollinear features. 
An additional feature selection step is required, using a custom function to eliminate featuers with high VIF values.

In [ ]:
def vif_feature_selection(X, thresh=10):
    '''Iterate through the features and calculate their respective values,
        continuously dropping the highest VIF features until all the features
        have VIF less than the threshold'''
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping {} at index: {}'.format(
                X.iloc[:, variables].columns[maxloc], str(maxloc)))
            del variables[maxloc]
            dropped = True

    print('Remaining variables: {}'.format(list(X.columns[variables])))
    return X.iloc[:, variables]

In [ ]:
# Apply the function to the dataframe with independent variables (X) and assign updated values as X.
X_vif = vif_feature_selection(X)

In [ ]:
X_vif.head(5)

##### From the above, the remaining variables are not multicollinear.

# Regression on Full data using OLS model

In [ ]:
Regression = sm.OLS(endog=y, exog=X_vif).fit()

In [ ]:
print(Regression.summary())

# Recap of outputs using this method
TODO...

# Overall takeaways
TODO...

# To read more on lasso and ridge regression.
https://towardsdatascience.com/ridge-and-lasso-regression-a-complete-guide-with-python-scikit-learn-e20e34bcbf0b